In [0]:
import openmeteo_requests

import pandas as pd
import requests_cache
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": [47.52, 50.5, 51.17, 40.46, 46.23, 45.1, 47.16, 41.87, 55.17, 52.13, 51.92, 39.4, 48.67],
	"longitude": [14.55, 4.47, 10.45, -3.75, 2.21, 15.2, 19.5, 12.56, 23.88, 5.29, 19.14, -8.22, 19.69],
	"hourly": ["temperature_2m", "wind_speed_10m", "shortwave_radiation"],
	"forecast_days": 1,
	"wind_speed_unit": "ms",
}
responses = openmeteo.weather_api(url, params=params)

all_rows = []   # <-- NEW: collect all dataframes here

# Process 13 locations
for response in responses:

    lat = response.Latitude()
    lon = response.Longitude()

    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(1).ValuesAsNumpy()
    hourly_shortwave_radiation = hourly.Variables(2).ValuesAsNumpy()

    hourly_data = {
        "lat": lat,
        "lon": lon,
        "timestamp": pd.date_range(
            start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
            end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
            freq=pd.Timedelta(seconds=hourly.Interval()),
            inclusive="left"
        ),
        "ssrd": hourly_shortwave_radiation,
         "wind_speed": hourly_wind_speed_10m,
        "temperature_c": hourly_temperature_2m
    }

    df = pd.DataFrame(hourly_data)
    all_rows.append(df)

# Combine all into ONE dataframe
full_table = pd.concat(all_rows, ignore_index=True)
full_table = full_table.sort_values(["timestamp", "lat", "lon"])

print(full_table)